In [1]:
"""Getting all relevent imports into the file"""
import re
import pandas as pd #imported pandas to use dataframes
import numpy as np #imported numpy for dynamic arrays
import matplotlib.pyplot as plt # imported pyplot to use with my visuals
import requests #imported to use webscrape
from requests import get #pulled get to use for scraping
from random import randint #pulled randint to use as a buffer for time
from time import sleep #pulled sleep to slow the crawl rate of the scraper
from bs4 import BeautifulSoup #pulled to webscrape

In [7]:
"""Pulling information from the actual IMDb website."""
pages = np.arange(1,10000,250)
title = []
movie_rating = []
year =  []
star_rating = []
gross = []
movie_genre = []
movie_time = []
votes = []

def scraper():
    for page in pages:
        page = requests.get('https://www.imdb.com/search/title/?release_date=1990-01-01,2020-12-31&user_rating=4.0,9.0&certificates=US%3AG,US%3APG,US%3APG-13,US%3AR,US%3ANC-17&countries=us&sort=alpha,asc&count=250&start='+ str(page) +'&ref_=adv_nxt')
        
        soup = BeautifulSoup(page.text, 'html.parser')
        
#pulling from lister-item mode-advanced pulls all the information for each of the movies on the page to easily pull 
#the Html tags as attributes of an item
        container = soup.findAll('div',class_ = 'lister-item mode-advanced') 
        """ for loop looking for each of the catagories in each of the containers"""
        for name in container:
            """nv is so we can seperate the gross income and seperate it from the votes set into their own """
            nv = name.find_all('span',attrs ={'name':'nv'})
            
            
            
            title.append(name.h3.a.text)
            movie_rating.append(name.p.span.text)
            year.append(name.h3.find('span',class_ = 'lister-item-year').text)
            
            if name.strong is not None:
                star_rating.append(float(name.strong.text))
            else:
                star_rating.append(0)
                
            gross_income = nv[1].text.strip('$\n\tM') if len(nv) > 1 else '0'
            gross.append(gross_income)
            votes_total = nv[0].text if len(nv) >1 else 0
            votes.append(votes_total)
            
            if name.p.find('span',class_ = 'genre') is not None:
                movie_genres = name.p.find('span',class_ ='genre').text#.strip(',\n\t" "').split(',')
                movie_genre.append(movie_genres)#list(map(lambda ele: ele.split(','),movie_genres))
            else:
                movie_genre.append('N/A')
            
            if name.p.find('span',class_ = 'runtime') is None:
                movie_time.append(0)
            else:
                movie_time.append(name.p.find('span', class_ = 'runtime').text)

In [12]:
"""Cleaning of the information. removing unneeded characters out of the set of information as well as setting up the dataframe"""
def Clean():
    
    df.loc[:,'Year'] = df['Year'].astype(str).str.strip('()TV Special Movie VideoGame TV–\I Short" "X')
    df.loc[:,'Movie Runtime'] = df['Movie Runtime'].astype(str).str.strip('min\," "').astype(int)
    df.loc[:,'Votes'] = df['Votes'].astype(str).str.replace(',','').astype(float)
    df.loc[:,'Gross Income']= df['Gross Income'].astype(str).str.replace('–'," ",True).astype(float)
    df.loc[:, 'Title'] = df['Title'].astype(str)
    df.loc[:,'Movie Genre'] = df['Movie Genre'].astype(str).str.replace(' ',"").str.replace("'","").str.strip('\n[]')
    
    
    

In [9]:
"""Pull and clean information to get ready for setting to dataframe"""
scraper()


In [15]:
"""Setting the information from above to a dataframe to manipulate and visualize"""
df = pd.DataFrame({'Title': title,'Movie Rating': movie_rating,'Year': year,
                   'Gross Income': gross,'Movie Genre': movie_genre, 'Movie Runtime': movie_time,
                    'IMDb Rating': star_rating,'Votes': votes})
Clean()

df.to_csv('All_movies_year_1990-2020.csv')